In [4]:
!pip install opencv-python


In [3]:
import os
image_dir = "C:\\Users\Dreamboy\Downloads\jeevikas\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"
files = os.listdir(image_dir)
print("Files in the directory:")
print(files)


Files in the directory:
['Cyst', 'Normal', 'Stone', 'Tumor']


In [5]:
import os
import cv2 
image_dir = "C:\\Users\Dreamboy\Downloads\jeevikas\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"
images = []
# Function to recursively search for image files in a directory
def read_images_from_dir(directory):
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if filename.endswith((".jpg", ".jpeg", ".png")):
                filepath = os.path.join(root, filename)
                image = cv2.imread(filepath)  
                images.append(image)
read_images_from_dir(image_dir)
if len(images) == 0:
    print("No images found in the specified directory.")
else:
    print("Number of images found:", len(images))

Number of images found: 12446


In [6]:
import os
import cv2
import hashlib

def remove_duplicates(files):
    unique_files = []
    file_hashes = set()

    for file in files:
        with open(file, "rb") as f:
            file_hash = hashlib.md5(f.read()).hexdigest()
        if file_hash not in file_hashes:
            unique_files.append(file)
            file_hashes.add(file_hash)

    return unique_files

def remove_corrupted_files(files):
    valid_files = []

    for file in files:
        try:
            image = cv2.imread(file)
            if image is not None:
                valid_files.append(file)
        except Exception as e:
            print(f"Error reading {file}: {e}")

    return valid_files

def clean_dataset(image_dir):
    image_files = []
    for root, dirs, files in os.walk(image_dir):
        for file in files:
            if file.lower().endswith((".jpg", ".jpeg", ".png")):
                image_files.append(os.path.join(root, file))

    # Remove duplicates
    unique_files = remove_duplicates(image_files)
    print(f"Number of unique files after removing duplicates: {len(unique_files)}")

    # Remove corrupted files
    valid_files = remove_corrupted_files(unique_files)
    print(f"Number of valid files after removing corrupted files: {len(valid_files)}")

    return valid_files
image_dir = "C:\\Users\Dreamboy\Downloads\jeevikas\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"
cleaned_files = clean_dataset(image_dir)


Number of unique files after removing duplicates: 11929
Number of valid files after removing corrupted files: 11929


In [7]:
import os
import cv2
import numpy as np

def normalize_resize_images(image_dir, target_size):
    image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith((".jpg", ".jpeg", ".png"))]
    preprocessed_images = []
    for file in image_files:
        # Read image
        image = cv2.imread(file)
        if image is not None:
            
            image = image.astype(np.float32) / 255.0
           
            image = cv2.resize(image, target_size)
            preprocessed_images.append(image)

    return preprocessed_images
image_dir = "C:\\Users\Dreamboy\Downloads\jeevikas\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"
target_size = (224, 224)  
preprocessed_images = normalize_resize_images(image_dir, target_size)


In [8]:
import os
import cv2
import numpy as np
dataset_dir = "C:\\Users\Dreamboy\Downloads\jeevikas\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"
images = []
labels = []
for class_name in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_dir):
        for filename in os.listdir(class_dir):
            image_path = os.path.join(class_dir, filename)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (224, 224))  
            image = image / 255.0 
            images.append(image)
            labels.append(class_name)
images = np.array(images)
labels = np.array(labels)


In [9]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
num_classes = 4  
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.summary()



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 24,113,284 (91.98 MB)

 Trainable params: 525,572 (2.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
label_dict = {label: idx for idx, label in enumerate(np.unique(labels))}
num_classes = len(label_dict)
labels_encoded = np.array([label_dict[label] for label in labels])
labels_onehot = to_categorical(labels_encoded, num_classes=num_classes)
X_train, X_val, y_train, y_val = train_test_split(images, labels_onehot, test_size=0.2, random_state=42)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 948s 3s/step - accuracy: 0.5267 - loss: 1.1422 - val_accuracy: 0.6835 - val_loss: 0.8085
Epoch 2/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 879s 3s/step - accuracy: 0.6938 - loss: 0.7757 - val_accuracy: 0.7100 - val_loss: 0.7644
Epoch 3/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 871s 3s/step - accuracy: 0.7599 - loss: 0.6306 - val_accuracy: 0.7904 - val_loss: 0.5081
Epoch 4/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 903s 3s/step - accuracy: 0.7922 - loss: 0.5377 - val_accuracy: 0.8526 - val_loss: 0.4024
Epoch 5/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 857s 3s/step - accuracy: 0.8407 - loss: 0.4268 - val_accuracy: 0.8036 - val_loss: 0.5000
Epoch 6/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 879s 3s/step - accuracy: 0.8481 - loss: 0.4079 - val_accuracy: 0.7900 - val_loss: 0.5179
Epoch 7/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 890s 3s/step - accuracy: 0.8742 - loss: 0.3532 - val_accuracy: 0.8353 - val_loss: 0.3902
Epoch 8/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 864s 3s/step - accuracy: 0.8892 - loss: 0.3109 - val_accu

In [30]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
X_test = X_test.astype(np.float32)
y_pred_prob = model.predict(X_test)

y_pred = np.argmax(y_pred_prob, axis=1)

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
y_test_numerical = label_encoder.fit_transform(y_test)
y_test_onehot = to_categorical(y_test_numerical)
loss, accuracy = model.evaluate(X_test, y_test_onehot)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report(np.argmax(y_test_onehot, axis=1), y_pred))
print("Confusion Matrix:")
print(confusion_matrix(np.argmax(y_test_onehot, axis=1), y_pred))


78/78 ━━━━━━━━━━━━━━━━━━━━ 178s 2s/step
78/78 ━━━━━━━━━━━━━━━━━━━━ 173s 2s/step - accuracy: 0.9574 - loss: 0.1865
Test Loss: 0.19468794763088226, Test Accuracy: 0.9461847543716431
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.96      0.95       765
           1       0.97      0.98      0.98      1015
           2       0.89      0.76      0.82       276
           3       0.91      0.97      0.94       434

    accuracy                           0.95      2490
   macro avg       0.93      0.92      0.92      2490
weighted avg       0.95      0.95      0.95      2490

Confusion Matrix:
[[734   2  17  12]
 [  2 992   6  15]
 [ 38  14 211  13]
 [  2  10   3 419]]


In [15]:
print("Data type of X_test:", X_test.dtype)
print("Data type of y_test:", y_test.dtype)


Data type of X_test: float64
Data type of y_test: <U6


In [20]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and transform labels
y_test_encoded = label_encoder.fit_transform(y_test)

# Convert to integers
y_test_int = y_test_encoded.astype(int)



In [27]:
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)



Shape of X_test: (2490, 224, 224, 3)
Shape of y_test: (2490,)


In [28]:
from sklearn.preprocessing import LabelEncoder

# Initialize label encoder
label_encoder = LabelEncoder()

# Convert class labels to integers
y_test_encoded = label_encoder.fit_transform(y_test)

# Now, convert y_test_encoded to one-hot encoded format
y_test_onehot = to_categorical(y_test_encoded)


In [29]:
# Convert X_test to float32
X_test = X_test.astype(np.float32)

# Convert y_test to numerical labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_test_numerical = label_encoder.fit_transform(y_test)

# Convert numerical labels to one-hot encoded format
from tensorflow.keras.utils import to_categorical
y_test_onehot = to_categorical(y_test_numerical)


In [34]:
import tensorflow as tf
model.save('C:\\Users\\Dreamboy\\Downloads\\jeevikas\\modeltrained.keras')

In [53]:
import numpy as np
import cv2
import tensorflow as tf
model = tf.keras.models.load_model(r'C:\Users\Dreamboy\Downloads\jeevikas\modeltrained.keras')
class_names = ['Cyst', 'Normal', 'Stone', 'Tumor']

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image at path {image_path} could not be loaded.")
    
    image = cv2.resize(image, (224, 224))
    image = image.astype(np.float32) / 255.0
    image = np.expand_dims(image, axis=0)
    
    return image

def predict_image(image):
    probabilities = model.predict(image)
    
    return probabilities

def analyze_results(probabilities):
    threshold = 0.5
    predicted_class_index = np.argmax(probabilities)
    predicted_class_name = class_names[predicted_class_index]
    predicted_probability = probabilities[0, predicted_class_index]
    
    if predicted_class_index == 1:
        print(f"The image indicates a normal kidney (Class {predicted_class_name})")
    else:
        if predicted_probability > threshold:
            print(f"The image indicates a kidney abnormality (Class {predicted_class_name}) with a probability of {predicted_probability:.2f}")
        else:
            print(f"The image indicates a normal kidney with a low probability of abnormality (Class {predicted_class_name}) with a probability of {predicted_probability:.2f}")

# Example usage
image_path = 'C:\\Users\\Dreamboy\\Downloads\\kidneyc.png'
try:
    image = preprocess_image(image_path)
    probabilities = predict_image(image)
    analyze_results(probabilities)
except Exception as e:
    print(f"Error: {e}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
The image indicates a kidney abnormality (Class Cyst) with a probability of 1.00
